# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'AQNSAD',
 'calculationPrice': 'close',
 'open': 456.1,
 'openTime': 1626872646066,
 'openSource': 'ificflao',
 'close': 457.93,
 'closeTime': 1660935115121,
 'closeSource': 'iiclofaf',
 'high': 474.4,
 'highTime': 1599194523036,
 'highSource': 'ua ieedlnrymceeid 5 1tp',
 'low': 455.27,
 'lowTime': 1667491842951,
 'lowSource': 'mde niaeet ydplceriu 51',
 'latestPrice': 463.12,
 'latestSource': 'Close',
 'latestTime': 'August 12, 2020',
 'latestUpdate': 1641878956772,
 'latestVolume': 172268,
 'iexRealtimePrice': 0,
 'iexRealtimeSize': 0,
 'iexLastUpdated': 0,
 'delayedPrice': 467.3,
 'delayedPriceTime': 1675959585423,
 'oddLotDelayedPrice': 459,
 'oddLotDelayedPriceTime': 1611292724235,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 458,
 'previousVolume': 47168921,
 'change': 14.93,
 'changePercent': 0.0337,
 'volume': 0,
 'iexMarketPerc

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

34.73

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,97.75,45.86,N/A
1,AAL,13.72,-1.7,N/A
2,AAP,159.16,28.52,N/A
3,AAPL,455.28,34.39,N/A
4,ABBV,95.61,20.32,N/A
...,...,...,...,...
500,YUM,95.20,27.5,N/A
501,ZBH,140.77,738.94,N/A
502,ZBRA,296.68,31.21,N/A
503,ZION,35.45,13.71,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,64.98,2478.19,N/A
1,FIS,145.45,1505,N/A
2,GPC,97.10,984.11,N/A
3,ZBH,140.77,738.94,N/A
4,HLT,87.98,600.76,N/A
5,STZ,180.10,599.64,N/A
6,HRB,15.44,548.8,N/A
7,GLW,33.14,262.91,N/A
8,IQV,165.07,246.49,N/A
9,KMI,15.17,211.73,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,64.98,2478.19,301
1,FIS,145.45,1505,134
2,GPC,97.10,984.11,201
3,ZBH,140.77,738.94,139
4,HLT,87.98,600.76,222
5,STZ,180.10,599.64,108
6,HRB,15.44,548.8,1269
7,GLW,33.14,262.91,591
8,IQV,165.07,246.49,118
9,KMI,15.17,211.73,1292


## Building a Better (and More Realistic) Momentum Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest and Taxes (EV/EBIT)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [18]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBIT
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['enterpriseValue']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [11]:
rv_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'EV/EBIT',
                'EV/EBIT Percentile',
                'EV/FCF',
                'EV/FCF Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        rv_dataframe = rv_dataframe.append(
                                        pd.Series([
                                                    symbol, 
                                                    data[symbol]['quote']['latestPrice'], 
                                                    'N/A', 
                                                    'Price-to-Earnings Ratio',
                                                    'PE Percentile',
                                                    'Price-to-Book Ratio',
                                                    'PB Percentile',
                                                    'EV/EBIT',
                                                    'EV/EBIT Percentile',
                                                    'EV/FCF',
                                                    'EV/FCF Percentile',
                                                    'EV/GP',
                                                    'EV/GP Percentile',
                                                    'RV Score'
                                                   ], 
                                                  index = rv_columns), 
                                        ignore_index = True)
        

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBIT,EV/EBIT Percentile,EV/FCF,EV/FCF Percentile,EV/GP,EV/GP Percentile,RV Score
